# Exercise 3: Message History & Memory
## Building Financial Advisors with Conversation Memory

In this notebook, we'll learn how to add conversation memory to our financial advisors so they can track portfolio discussions and remember previous conversations about financial goals.

## Learning Objectives
- Understand LangChain message history and memory concepts
- Implement conversation memory for financial discussions
- Create a financial goal tracker that remembers context
- Use MessagesPlaceholder, HumanMessage, and AIMessage
- Build persistent chat history for portfolio management

Let's build memory-enabled financial advisors! 🧠💰

## Setup and Imports

First, let's install and import the necessary libraries for working with message history and memory.

In [1]:
# Install required packages
!uv add langchain langchain-openai python-dotenv

Resolved 383 packages in 4ms
Audited 214 packages in 0.12ms


In [2]:
# Import necessary libraries
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
import os
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()

# Initialize ChatOpenAI
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,  # Slightly lower for consistent financial advice
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✓ Setup complete!")
print("✓ Memory components imported!")
print("✓ Ready to build memory-enabled financial advisors!")

✓ Setup complete!
✓ Memory components imported!
✓ Ready to build memory-enabled financial advisors!


## Part 1: Understanding Message History

Let's start by understanding how to manually create and manage message history using HumanMessage and AIMessage.

In [3]:
# Create a simple conversation history
conversation_history = [
    SystemMessage(content="You are a helpful financial advisor specializing in portfolio management."),
    HumanMessage(content="Hi, I'm 30 years old and want to start investing. I have $50,000 to invest."),
    AIMessage(content="Great! At 30, you have a good time horizon for investing. With $50,000, I'd recommend a diversified approach. What are your main financial goals - retirement, house purchase, or general wealth building?"),
    HumanMessage(content="I want to save for retirement and maybe buy a house in 5-7 years."),
    AIMessage(content="Perfect! For dual goals like retirement and house purchase, I'd suggest splitting your investments. For the house fund (shorter term), consider safer investments like bonds or CDs. For retirement (longer term), you can be more aggressive with stocks and index funds.")
]

# Use the conversation history in a new query
new_message = HumanMessage(content="What specific percentage should I allocate to each category?")
conversation_history.append(new_message)

# Get response with full context
response = llm.invoke(conversation_history)
print("AI Response with Context:")
print(response.content)

# Add the AI response to history
conversation_history.append(AIMessage(content=response.content))
print(f"\n✓ Conversation now has {len(conversation_history)} messages")

AI Response with Context:
A common rule of thumb is the "100 minus your age" rule. Since you're 30, you could consider allocating around 70% to stocks (for long-term growth) and 30% to bonds or other safer investments (for short-term stability). You can adjust this allocation based on your risk tolerance and financial goals.

✓ Conversation now has 7 messages


## Part 2: Using MessagesPlaceholder for Dynamic History

Now let's use MessagesPlaceholder to create templates that can dynamically include conversation history.

In [4]:
# Create a chat template with MessagesPlaceholder
portfolio_advisor_template = ChatPromptTemplate.from_messages([
    ("system", """You are an expert portfolio advisor with deep knowledge of investment strategies. 
    You help clients build and manage their investment portfolios based on their goals, risk tolerance, and time horizon.
    
    Always consider the conversation history to provide contextual and personalized advice.
    Be specific with allocation percentages and investment recommendations."""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

print("✓ Portfolio advisor template created with MessagesPlaceholder")

# Test the template
test_history = [
    HumanMessage(content="I'm 25 years old and want to start investing $1000 per month."),
    AIMessage(content="That's excellent! At 25, you have a 40-year investment horizon for retirement. With $1000 monthly, you could build substantial wealth. What's your risk tolerance - conservative, moderate, or aggressive?"),
    HumanMessage(content="I'd say I'm moderate risk tolerance.")
]

# Format the prompt with history
formatted_prompt = portfolio_advisor_template.format_prompt(
    chat_history=test_history,
    input="Based on our discussion, what specific portfolio allocation would you recommend?"
)

# Get response
response = llm.invoke(formatted_prompt.to_messages())
print("\nPortfolio Advisor Response:")
print(response.content)

✓ Portfolio advisor template created with MessagesPlaceholder

Portfolio Advisor Response:
Given your moderate risk tolerance and long investment horizon, a diversified portfolio of stocks and bonds would be suitable. Here's a recommended allocation:

- 70% in a diversified portfolio of low-cost index funds or ETFs that track the overall stock market (such as S&P 500 or total stock market index).
- 30% in a diversified portfolio of bond funds to provide stability and lower overall portfolio volatility.

This allocation aims to balance growth potential with risk management. Regularly review and adjust your portfolio as needed to stay aligned with your financial goals.


## Part 3: Building a Financial Goal Tracker with Memory

Let's create a comprehensive financial goal tracker that remembers client information and tracks progress over time.

In [5]:
class FinancialGoalTracker:
    """A financial advisor that remembers client information and tracks financial goals."""
    
    def __init__(self, llm):
        self.llm = llm
        self.conversation_history = []
        self.client_profile = {}
        self.financial_goals = []
        
        # Create the advisor template
        self.template = ChatPromptTemplate.from_messages([
            ("system", """You are a personal financial advisor with access to the client's complete history.
            
            Client Profile: {client_profile}
            Financial Goals: {financial_goals}
            
            Use this information to provide personalized, contextual advice. Always reference previous discussions
            and track progress toward their goals. Be encouraging and specific in your recommendations."""),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}")
        ])
    
    def add_client_info(self, key, value):
        """Add or update client profile information."""
        self.client_profile[key] = value
        print(f"✓ Updated client profile: {key} = {value}")
    
    def add_financial_goal(self, goal, target_amount, target_date):
        """Add a new financial goal."""
        goal_info = {
            "goal": goal,
            "target_amount": target_amount,
            "target_date": target_date,
            "date_added": datetime.now().strftime("%Y-%m-%d")
        }
        self.financial_goals.append(goal_info)
        print(f"✓ Added financial goal: {goal} - ${target_amount:,} by {target_date}")
    
    def chat(self, message):
        """Have a conversation with the financial advisor."""
        # Prepare the prompt
        prompt = self.template.format_prompt(
            client_profile=str(self.client_profile),
            financial_goals=str(self.financial_goals),
            chat_history=self.conversation_history,
            input=message
        )
        
        # Get response
        response = self.llm.invoke(prompt.to_messages())
        
        # Update conversation history
        self.conversation_history.append(HumanMessage(content=message))
        self.conversation_history.append(AIMessage(content=response.content))
        
        return response.content
    
    def get_conversation_summary(self):
        """Get a summary of the conversation."""
        return {
            "client_profile": self.client_profile,
            "financial_goals": self.financial_goals,
            "conversation_length": len(self.conversation_history),
            "last_updated": datetime.now().strftime("%Y-%m-%d %H:%M")
        }

# Create a financial goal tracker instance
advisor = FinancialGoalTracker(llm)
print("✓ Financial Goal Tracker created!")

✓ Financial Goal Tracker created!


## Part 4: Demonstrating the Financial Goal Tracker

Let's see our Financial Goal Tracker in action with a realistic scenario!

In [6]:
# Let's create a complete example with Sarah, a 28-year-old software engineer
print("=== Setting up Sarah's Financial Profile ===")

# Add client information
advisor.add_client_info("name", "Sarah Chen")
advisor.add_client_info("age", 28)
advisor.add_client_info("occupation", "Software Engineer")
advisor.add_client_info("annual_income", 95000)
advisor.add_client_info("current_savings", 15000)
advisor.add_client_info("monthly_savings", 2000)

# Add financial goals
advisor.add_financial_goal("Emergency Fund", 30000, "2025-12-31")
advisor.add_financial_goal("House Down Payment", 80000, "2028-01-01")
advisor.add_financial_goal("Retirement", 1000000, "2058-01-01")

print("\n=== Starting Conversation ===")
# First conversation
response1 = advisor.chat("Hi! I'm Sarah. I've been saving money but I'm not sure how to invest it wisely. Can you help me create an investment strategy?")
print("Advisor:", response1[:200] + "..." if len(response1) > 200 else response1)

print("\n" + "="*50)
# Second conversation - the advisor should remember everything
response2 = advisor.chat("I'm particularly worried about market volatility. Given my timeline for buying a house, should I keep that money in safer investments?")
print("Advisor:", response2[:200] + "..." if len(response2) > 200 else response2)

print("\n" + "="*50)
# Third conversation - asking about progress
response3 = advisor.chat("It's been 6 months since we started. I've been following your advice and now have $18,000 saved. Should I adjust my strategy?")
print("Advisor:", response3[:200] + "..." if len(response3) > 200 else response3)

=== Setting up Sarah's Financial Profile ===
✓ Updated client profile: name = Sarah Chen
✓ Updated client profile: age = 28
✓ Updated client profile: occupation = Software Engineer
✓ Updated client profile: annual_income = 95000
✓ Updated client profile: current_savings = 15000
✓ Updated client profile: monthly_savings = 2000
✓ Added financial goal: Emergency Fund - $30,000 by 2025-12-31
✓ Added financial goal: House Down Payment - $80,000 by 2028-01-01
✓ Added financial goal: Retirement - $1,000,000 by 2058-01-01

=== Starting Conversation ===
Advisor: Hi Sarah! I'm glad to hear that you're thinking about investing your savings wisely. Given your financial goals and current situation, we can tailor an investment strategy that aligns with your object...

Advisor: That's a valid concern, Sarah. Market volatility can impact the value of your investments, especially in the short term. Since your house down payment goal has a specific timeline of January 1, 2028, ...

Advisor: That's great

In [7]:
# Let's examine what the advisor remembers about Sarah
print("=== Conversation Summary ===")
summary = advisor.get_conversation_summary()
print(f"Client Profile: {summary['client_profile']}")
print(f"Financial Goals: {len(summary['financial_goals'])} goals tracked")
print(f"Conversation Length: {summary['conversation_length']} messages")
print(f"Last Updated: {summary['last_updated']}")

print("\n=== Full Conversation History ===")
for i, message in enumerate(advisor.conversation_history):
    speaker = "Human" if isinstance(message, HumanMessage) else "AI"
    content = message.content[:100] + "..." if len(message.content) > 100 else message.content
    print(f"{i+1}. {speaker}: {content}")

print("\n=== Testing Memory - New Question ===")
# Test that the advisor remembers the context
response4 = advisor.chat("Quick question - what was my current savings amount when we first started talking?")
print("Advisor:", response4)

=== Conversation Summary ===
Client Profile: {'name': 'Sarah Chen', 'age': 28, 'occupation': 'Software Engineer', 'annual_income': 95000, 'current_savings': 15000, 'monthly_savings': 2000}
Financial Goals: 3 goals tracked
Conversation Length: 6 messages
Last Updated: 2025-07-06 08:12

=== Full Conversation History ===
1. Human: Hi! I'm Sarah. I've been saving money but I'm not sure how to invest it wisely. Can you help me crea...
2. AI: Hi Sarah! I'm glad to hear that you're thinking about investing your savings wisely. Given your fina...
3. Human: I'm particularly worried about market volatility. Given my timeline for buying a house, should I kee...
4. AI: That's a valid concern, Sarah. Market volatility can impact the value of your investments, especiall...
5. Human: It's been 6 months since we started. I've been following your advice and now have $18,000 saved. Sho...
6. AI: That's great progress, Sarah! Saving $18,000 in just 6 months is commendable and puts you well on tr...

=== 

## Part 5: Advanced Memory Techniques

Let's explore different types of memory that LangChain provides for more sophisticated conversation handling.

In [8]:
# 1. ConversationBufferMemory - Keeps all messages in memory
print("=== ConversationBufferMemory ===")
buffer_memory = ConversationBufferMemory(return_messages=True)

# Add some conversation to memory
buffer_memory.chat_memory.add_user_message("I want to invest $10,000 in index funds.")
buffer_memory.chat_memory.add_ai_message("Great choice! Index funds offer diversification and low fees. I'd recommend a mix of total stock market and international funds.")
buffer_memory.chat_memory.add_user_message("What percentage should I allocate to each?")
buffer_memory.chat_memory.add_ai_message("For a balanced approach, consider 70% total stock market and 30% international funds.")

# Check memory contents
print("Buffer Memory Contents:")
for message in buffer_memory.chat_memory.messages:
    speaker = "Human" if isinstance(message, HumanMessage) else "AI"
    print(f"  {speaker}: {message.content}")

print(f"\nTotal messages in buffer: {len(buffer_memory.chat_memory.messages)}")

print("\n" + "="*50)

# 2. ConversationSummaryMemory - Summarizes old conversations
print("=== ConversationSummaryMemory ===")
summary_memory = ConversationSummaryMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=100  # Keep summaries short for demo
)

# Add the same conversation
summary_memory.chat_memory.add_user_message("I want to invest $10,000 in index funds.")
summary_memory.chat_memory.add_ai_message("Great choice! Index funds offer diversification and low fees. I'd recommend a mix of total stock market and international funds.")
summary_memory.chat_memory.add_user_message("What percentage should I allocate to each?")
summary_memory.chat_memory.add_ai_message("For a balanced approach, consider 70% total stock market and 30% international funds.")

# Add more conversation to trigger summarization
summary_memory.chat_memory.add_user_message("Should I rebalance my portfolio quarterly?")
summary_memory.chat_memory.add_ai_message("Quarterly rebalancing is good practice. It helps maintain your target allocation without over-trading.")

print("Summary Memory Contents:")
print(f"Summary: {summary_memory.buffer}")
print(f"Total messages: {len(summary_memory.chat_memory.messages)}")

print("\n✓ Memory techniques demonstrated!")
print("✓ Buffer Memory: Keeps all messages (good for short conversations)")
print("✓ Summary Memory: Summarizes old messages (good for long conversations)")

=== ConversationBufferMemory ===
Buffer Memory Contents:
  Human: I want to invest $10,000 in index funds.
  AI: Great choice! Index funds offer diversification and low fees. I'd recommend a mix of total stock market and international funds.
  Human: What percentage should I allocate to each?
  AI: For a balanced approach, consider 70% total stock market and 30% international funds.

Total messages in buffer: 4

=== ConversationSummaryMemory ===
Summary Memory Contents:
Summary: 
Total messages: 6

✓ Memory techniques demonstrated!
✓ Buffer Memory: Keeps all messages (good for short conversations)
✓ Summary Memory: Summarizes old messages (good for long conversations)


/tmp/ipykernel_9763/1144786237.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory(return_messages=True)
/tmp/ipykernel_9763/1144786237.py:23: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(


## Part 6: Interactive Exercise

Now it's your turn! Create your own financial advisor conversation and experiment with memory.

In [9]:
# Create your own financial advisor client!
# TODO: Customize this section with your own client information

# 1. Create a new advisor instance
my_advisor = FinancialGoalTracker(llm)

# 2. Add your client information (customize these values)
my_advisor.add_client_info("name", "Your Name Here")
my_advisor.add_client_info("age", 25)  # Change to your age
my_advisor.add_client_info("occupation", "Your Job")
my_advisor.add_client_info("annual_income", 60000)  # Change to your income
my_advisor.add_client_info("current_savings", 10000)  # Change to your savings

# 3. Add financial goals (customize these)
my_advisor.add_financial_goal("Emergency Fund", 20000, "2025-06-01")
my_advisor.add_financial_goal("Vacation", 5000, "2025-12-01")
# Add more goals as needed

# 4. Start your conversation
print("=== Your Financial Advisor Conversation ===")
response = my_advisor.chat("Hi! I'm just getting started with investing. What should I focus on first?")
print("Advisor:", response)

print("\n" + "="*50)
print("Continue the conversation by adding more advisor.chat() calls below!")
print("Try asking about:")
print("- Specific investment recommendations")
print("- How to prioritize your goals")
print("- Risk tolerance assessment")
print("- Portfolio rebalancing strategies")

# Add your own questions here:
# response2 = my_advisor.chat("Your question here")
# print("Advisor:", response2)

✓ Updated client profile: name = Your Name Here
✓ Updated client profile: age = 25
✓ Updated client profile: occupation = Your Job
✓ Updated client profile: annual_income = 60000
✓ Updated client profile: current_savings = 10000
✓ Added financial goal: Emergency Fund - $20,000 by 2025-06-01
✓ Added financial goal: Vacation - $5,000 by 2025-12-01
=== Your Financial Advisor Conversation ===
Advisor: Hello! As you're just getting started with investing, it's great to see you taking this step towards building your financial future. Given your current financial situation and goals, here are a few recommendations to help you focus on first:

1. **Emergency Fund**: Since you have a goal of building an emergency fund with a target amount of $20,000 by June 2025, I recommend prioritizing this goal. Aim to set aside a portion of your monthly income towards your emergency fund until you reach your target amount. This fund will provide you with a financial safety net in case of unexpected expenses

## Summary & Best Practices

### What You've Learned 🎓

In this notebook, you've mastered the fundamentals of message history and memory in LangChain:

1. **Manual Message History**: Creating conversation flows with `HumanMessage` and `AIMessage`
2. **MessagesPlaceholder**: Dynamic templates that can include conversation history
3. **Financial Goal Tracker**: A comprehensive class that tracks client profiles and conversation history
4. **Memory Types**: Buffer memory vs. summary memory for different use cases
5. **Persistent Context**: How to maintain context across multiple conversation turns

### Key Concepts Mastered 🔑

- **Message Types**: SystemMessage, HumanMessage, AIMessage
- **Template Design**: Using MessagesPlaceholder for dynamic conversations
- **Memory Management**: Choosing between buffer and summary memory
- **Context Persistence**: Maintaining client information across conversations
- **Conversation Flow**: Building natural, contextual financial advisory conversations

### Best Practices for Memory-Enabled Chatbots 💡

1. **Choose the Right Memory Type**
   - Buffer Memory: Short conversations, need full context
   - Summary Memory: Long conversations, want to save tokens
   - Custom Memory: Specific business logic requirements

2. **Structure Your Data**
   - Keep client profiles organized
   - Track goals and progress separately
   - Use timestamps for tracking progress

3. **Design for Context**
   - Include relevant information in system messages
   - Use conversation history to personalize responses
   - Reference previous discussions naturally

4. **Handle Memory Efficiently**
   - Monitor conversation length
   - Implement memory limits when needed
   - Consider summarization for long conversations

### Next Steps 🚀

- Experiment with different memory configurations
- Build more complex financial tracking systems
- Integrate with databases for persistent storage
- Add features like goal progress tracking and reminders

Great work! You now have the foundation to build sophisticated conversational AI systems with memory! 🎉